In [247]:
import numpy as np
import pandas as pd
import itertools
import pickle
import datetime
from tqdm.notebook import tqdm

#%matplotlib inline

In [240]:
%run get_five_letter_words

Letters in 5-letter words:
a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, x, y, z, å, æ, ø
Number of 5-letter words: 8,195


In [657]:
%run wordle

In [658]:
wl = WordleLookup()

wl.calculate_full_lookup(verbose = True)


Guess number 0    of 8195
Guess number 100  of 8195
Guess number 200  of 8195
Guess number 300  of 8195
Guess number 400  of 8195
Guess number 500  of 8195
Guess number 600  of 8195
Guess number 700  of 8195
Guess number 800  of 8195
Guess number 900  of 8195
Guess number 1000 of 8195
Guess number 1100 of 8195
Guess number 1200 of 8195
Guess number 1300 of 8195
Guess number 1400 of 8195
Guess number 1500 of 8195
Guess number 1600 of 8195
Guess number 1700 of 8195
Guess number 1800 of 8195
Guess number 1900 of 8195
Guess number 2000 of 8195
Guess number 2100 of 8195
Guess number 2200 of 8195
Guess number 2300 of 8195
Guess number 2400 of 8195
Guess number 2500 of 8195
Guess number 2600 of 8195
Guess number 2700 of 8195
Guess number 2800 of 8195
Guess number 2900 of 8195
Guess number 3000 of 8195
Guess number 3100 of 8195
Guess number 3200 of 8195
Guess number 3300 of 8195
Guess number 3400 of 8195
Guess number 3500 of 8195
Guess number 3600 of 8195
Guess number 3700 of 8195
Guess number

In [642]:
print(wl.words[:2])
r = [2,2,0,2,1]
v1 = wl._word2vec(wl.words[0])
v2 = wl._word2vec(wl.words[1])

print(v1,v2)
guess = np.array(v1)
answers = np.array([v1,v2])
r = wl.get_responses(guess, answers)
print(r)
n = wl._responses2num(r)
print(n)

print(2*3**4 + 2*3**3 + 0*3**2 + 1*3**1 + 1*3**0)

['abbed', 'abede']
[0 1 1 4 3] [0 1 4 3 4]
[[2 2 2 2 2]
 [2 2 0 1 1]]
[242 220]
220


##### Best first guess
1. For each word combination, get the response pattern for lookup
2. For each word, get the probabiity $p_i$ of each pattern and calculate the expected information as $$ I =  -\sum_i p_i \log_2(p_i)$$ 
This is the list for the initial guess
3.  After the first guess and response, only a subset of words is left. Find the next best guess as follows: For each word in the total list, get the probability %p_i% of each pattern (given the subset of words) and calculate again the expected information. This is the next best guess.

##### Best guess considering two guesses
4. Suppose "sarte" is chosen as the first word. Calculate the probability $p_i$ of each pattern_1. Each pattern_1 leads to a subset of words. For each pattern_1, run through all words (in the full list) and calculate probability $q_j$ for each pattern_2 (given the subset of words). The probability for getting the pattern combination pattern_1 + pattern_2 is $p_i q_j$. Hence $$ I = -\sum_{i,j} p_iq_j\log_2(p_iq_j) $$
where $i$ runs through all pattern_1 and $j$ runs through all pattern_2

In [346]:
def find_rows_with_duplicates(a):
    a_sorted = np.sort(a, axis=-1)
    return ~(a_sorted[...,1:] != a_sorted[...,:-1]).all(-1)

def get_response(
    guesses:np.ndarray,
    correct:np.ndarray) -> np.ndarray:
    """_summary_

    Args:
        guess (np.ndarray): Guess array
        correct (np.ndarray): Correct array

    Returns:
        _type_: _description_
    """
    
    assert guesses.ndim == 2 
    corrects = np.tile(correct,(guesses.shape[0],1))
    
    # First put greens
    green = (guesses == corrects)
    
    # Remove greens
    guesses[green] = -1 # Dummy
    corrects[green] = -2 # Not same as guesses dummy
    
    # Yellows
    yellow = np.isin(guesses, corrects)
    
    have_duplicate_letters = find_rows_with_duplicates(guesses)
    
    
    
    for row, (guess_has_duplicate_letters, guess, correct) in enumerate(zip(have_duplicate_letters, guesses, corrects)):
        if guess_has_duplicate_letters:
            # Get how many times each letter in guess should be colored
            # from the number of times that letter occure in the correct
            # Note: If greens already marked and ignored at this point
            correct_values, correct_counts = np.unique(correct, return_counts=True)
            correct_value_counts = {k:v for k,v in zip(correct_values,correct_counts)}
            guess_yellow_count = {g:correct_value_counts[g] for g in guess if g in correct_value_counts}
            
            for value, nb_occurences in guess_yellow_count.items():
                # Find the indexes after the first "nb_occurences" occurences of the value in guess
                # and remove the yellow flag from here
                idx = np.argwhere(np.isin(guess,value)).ravel()[nb_occurences:]
                yellow[row,idx] = False
    
    response = 2*green + 1*yellow
    
    return response

N = 1
for _ in range(N):
    guesses   = np.tile(np.array([1,1,0,1,5]),(N,1))
    correct   = np.array( [1,2,1,10,2])
    get_response(guesses,correct)
    
    if _ % 100 == 0:
        print(_/N)

0.0


In [604]:
def _flatten(t):
    return [item for sublist in t for item in sublist]

def _find_rows_with_duplicates(a):
    a_sorted = np.sort(a, axis=-1)
    return list(~(a_sorted[...,1:] != a_sorted[...,:-1]).all(-1))

def responses2num(responses):
    # Convert response to number by using base 3 (gray, yellow, green)
    return np.sum([responses[:,idx]*(3**(5-idx-1)) for idx in range(5)], axis = 0)

def nums2responses(nums):
    # Convert number to response array by using base 3 (gray, yellow, green)
    return np.array([[int(char) for char in np.base_repr(num,3).zfill(5)] for num in nums])

def get_responses(guess: np.ndarray, answers:np.ndarray) -> np.ndarray:
    """Get responses from a single guess on multiple answers.

    Args:
        guess (np.ndarray): The guess as numbers
        answers (np.ndarray): Array of answers to return responses for.
        Expected to have one guess for each row.

    Returns:
        np.ndarray: The responses as same size as answers.
            0: black (no match), 1: yellow (correct letter, wrong position)
            2: green (correct letter and position)
    """
    
    # Do not change the answers globally
    answers = answers.copy()
    
    # Expand guesses to same size as answers, to prepare for removal of letters
    guesses = np.tile(guess,(answers.shape[0],1))
        
    # First put greens
    green = (guesses == answers)

    # Remove greens from guess and answers by putting dummy
    guesses[green] = -1 # Dummy
    answers[green] = -2 # Not same as guesses dummy

    # Add green to response
    responses = 2*green
    
    # Add yellows depending on duplicated letters in guess
    guess_has_no_duplicates = len(guess) == len(set(guess))
    if guess_has_no_duplicates:
        # This is simple. Add the yellows by checking if guess letter is in answer
        yellow = np.isin(guesses, answers)
        responses += 1*yellow
        return responses
    else: # If t
        
        # Make yellows iteratively until no more matches are found
        # Find rows (words) that have still duplicated letters after green letters are removed
        all_rows = np.array(range(answers.shape[0]))
        check_rows = _find_rows_with_duplicates(guesses)
        while check_rows:
            intersects = [np.intersect1d(g, a,return_indices=True) for g,a in zip(guesses[check_rows],answers[check_rows])]
            idx_guess = [list(intersect[1]) for intersect in intersects]
            
            # Note, the output from intersect1d gives each row in the subset all_rows[check_rows]
            # Make sure "rows" below refer to the global row number
            rows = _flatten([[r]*len(g) for r,g in zip(all_rows[check_rows],idx_guess)])
            if rows: # If no rows found, continue and exit in next loop 
                idx_answer = [list(intersect[2]) for intersect in intersects]
                idx_guess = _flatten(idx_guess)
                idx_answer = _flatten(idx_answer)
                responses[rows,idx_guess] = 1 # Assign yellow
                guesses[rows,idx_guess] = -10 # Remove from guess after yellow
                answers[rows,idx_answer] = -11 # Remove from answer after yellow
                
            # Check the rows that has been assigned a yellow again for matches
            check_rows = list(set(rows))
            
        return responses
    
guess   = np.array( [1,1,3,1,5])
answers = np.array([[1,2,1,2,1],
                    [2,2,1,8,0]])

N = 5
#answers = np.random.randint(0,5,(N,5))
#guesses = np.random.randint(0,5,(N,5))
print(guess)
print(answers)
get_responses(guess, answers)

#R = np.zeros((N,N)).astype('int16')
# Matrix with index guess, answer (same guess in same row)
# for row, guess in enumerate(guesses):
#     responses = get_responses(guess, answers)
#     r = responses2num(responses)
#     R[row,:] = r


[1 1 3 1 5]
[[1 2 1 2 1]
 [2 2 1 8 0]]


array([[2, 1, 0, 1, 0],
       [1, 0, 0, 0, 0]])

In [577]:
np.unique(R, return_counts = True)

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 162, 163, 164, 165, 166, 167, 168, 169,
        170, 171, 172, 173, 174, 175, 176, 177, 178

In [557]:
2**8

256

In [477]:
list(range(4,-1,-1))

[4, 3, 2, 1, 0]

In [126]:
def flatten(t):
    return [item for sublist in t for item in sublist]

def ignore_extra_letters(guesses, correct):
    # Use only the first n occurences of a letter in each guess, where n is
    # the number of occurences of that letter in the correct guess
    correct_vals, correct_counts = np.unique(correct, return_counts=True)
    remove_cols = []
    remove_rows = []
    for correct_val, correct_count in zip(correct_vals, correct_counts):
        idx = np.array(np.where(guesses == correct_val)).T
        col_idx = np.split(idx[:,1], np.unique(idx[:, 0], return_index=True)[1][1:])
        remove_cols += [col_idx[correct_count:].tolist() for col_idx in col_idx]
        remove_rows += [[n]*len(c) for n,c in enumerate(remove_cols)]
        
    guesses[flatten(remove_rows),flatten(remove_cols)] = -1 # dummy < 0
    return guesses

def get_response(guesses, correct):
    
    # Get green (correct letter and position)
    green = (guesses == correct)
    
    # Remove the entries where green
    correct = np.tile(correct,[len(guesses),1])
    guesses[green] = -1 # Dummy
    correct[green] = -2 # Not same as guesses dummy
    
    # TODO: this function ignores the later letters
    # But it should first use the green letters, and then remove
    # the later letters.
    # guesses = ignore_extra_letters(guesses, correct)
    
    # Get yellow (correct letter but not position)
    yellow = (np.isin(guesses,correct) & ~green)
    
    # Get response
    response = green*2 + yellow*1
    return guesses,correct,green

guess =   np.array([[1, 3, 8, 3, 3],
                    [1, 5, 3, 10, 7]])
correct = np.array( [5, 5, 3, 7, 3])
yellow = np.array([[0,1,0,0,0],
                   [0,0,0,0,1]])
guesses,correct,green = get_response(guess, correct)
guesses,correct

(array([[ 1,  3,  8,  3, -1],
        [ 1, -1, -1, 10,  7]]),
 array([[ 5,  5,  3,  7, -2],
        [ 5, -2, -2,  7,  3]]))

In [134]:
_correct = correct.copy()
_response = np.zeros_like(_correct)
for col in range(5):
       row_isin = np.isin(_correct[:,col], guesses)
       vals_isin = _correct[row_isin,col]
       
       # Make response yellow
       _response[row_isin,col] = 1
       
       # Remove from correct
       _correct[row_isin,col] = -100
       
       print(col,row_isin,vals_isin)
       #if any(row_isin):
       #       break

guesses,correct,_correct, _response

0 [False False] []
1 [False False] []
2 [ True False] [3]
3 [ True  True] [7 7]
4 [False  True] [3]


(array([[ 1,  3,  8,  3, -1],
        [ 1, -1, -1, 10,  7]]),
 array([[ 5,  5,  3,  7, -2],
        [ 5, -2, -2,  7,  3]]),
 array([[   5,    5, -100, -100,   -2],
        [   5,   -2,   -2, -100, -100]]),
 array([[0, 0, 1, 1, 0],
        [0, 0, 0, 1, 1]]))

In [404]:
class WordleBrain():
    def __init__(self, words):
        self._assert_words(words)

        self.words = words
        self._let2num, self._num2let = self._get_letter_map()
        self.X = self._words2mat(self.words)
        self.nb_guesses = 0
        # Reduced X
        self.X_reduced = self.X
        
    def _get_letter_map(self):
        letters = sorted(list(set("".join(self.words))))
        let2num = {l:n for n,l in enumerate(letters)}
        num2let = {n:l for l,n in let2num.items()}
        return let2num, num2let

    def _assert_words(self, words):
        # Assert length = 5
        assert all([len(w) == 5 for w in words]), "All words must have length 5"

        # Assert case
        assert all([w.lower() == w for w in words]), "All words must be lower case"

        # Assert used letters
        only_allowed = "abcdefghijklmnopqrstuvxyzæøå"
        used = set("".join(words))
        assert all([u in only_allowed for u in used]), "Only allowed letters are {only_allowed}"

    def _word2vec(self, word):
            return np.array([self._let2num[l] for l in word]).astype('int8')
    
    def _vec2word(self, vec):
        return "".join([self._num2let[n] for n in vec])
    
    def _words2mat(self, words):
        return np.array([self._word2vec(w) for w in words]).astype('int8')
    
    def _mat2words(self,mat):
        return [self._vec2word(vec) for vec in mat]
    
    def reduce_vecs(self, X, guess, response):
        assert len(guess) == 5, "Guess must have length 5"
        assert len(response) == 5, "Response must hav length 5"
        assert all([r in [0,1,2] for r in response]), "Response must be either 0 (gray), 1 (orange), 2 (green)"
        guess = list(guess)

        double_response = [r for g,r in zip(guess, response) if guess.count(g) > 1]
        doubles_all_gray = all([r == 0 for r in double_response])
        doubles_all_green = all([r == 2 for r in double_response])
        safely_ignore_doubles = doubles_all_gray or doubles_all_green
        
        mask = np.array([True]*len(X))
        if safely_ignore_doubles:
            green_idx  = [idx for idx,r in enumerate(response) if r == 2]
            orange_idx = [idx for idx,r in enumerate(response) if r == 1]
            gray_idx   = [idx for idx,r in enumerate(response) if r == 0]
            green_guesses = np.array(guess)[green_idx]
            orange_guesses = np.array(guess)[orange_idx]
            gray_guesses = np.array(guess)[gray_idx]
            
            # Green: all letters must be in correct position
            mask &= np.all(X[:,green_idx] == green_guesses, axis = 1)
            
            # Orange: Word must contain all orange letters, but not in the guessed positions
            contains_all_orange = np.sum(np.isin(X,orange_guesses), axis = 1) == len(orange_guesses)
            letters_not_at_orange = np.all(X[:,orange_idx] != orange_guesses, axis = 1)
            mask &= contains_all_orange & letters_not_at_orange
            
            # Gray: Word must not contain letters
            contains_any_gray = np.any(np.isin(X, gray_guesses), axis = 1)
            mask &= ~contains_any_gray
            
        else: # Be aware that some letters may occure twice. 
            # The below code would also apply for no double letters
            # Iterate over each guess letter g and response r, from left to right
            colored_letters = []
            for idx, (g,r) in enumerate(zip(guess,response)):
                if r == 2: # Green: Correct letter and position
                    colored_letters.append(g)
                    mask &= X[:,idx] == g
                else: # If orange or gray
                    # Letter should not be in this position
                    mask &= X[:,idx] != g 
                    if r == 1: # Orange: Correct letter but wrong position
                        colored_letters.append(g)
                        if g in colored_letters:
                            # The letter must occure at least as many times as it is collered
                            mask &= np.sum(X == g, axis = 1) >= colored_letters.count(g)
                        else:
                            # Letter must be in word at least once
                            mask &= np.any(X == g, axis = 1)
                    elif r == 0: # Gray: Wrong letter
                        if g in colored_letters:
                            # The letter must occure exactly as many times as it is collered
                            mask &= np.sum(X == g, axis = 1) == colored_letters.count(g)
                        else:
                            # The letter cannot be in the word
                            mask &= ~np.any(X == g, axis = 1)
                            
        return X[mask]
    
    def get_information(self, X_reduced, guess):
        nb_all_words = len(X_reduced)
        information = 0
        # For all patterns
        for response in itertools.product(*[[0,1,2] for _ in range(5)]):
            nb_reduced = len(self.reduce_vecs(X_reduced, guess, response))
            prob = nb_reduced / nb_all_words
            information += - prob * np.log2(prob) if prob != 0 else 0

        return information
    
    def get_all_informations(self, X, X_reduced, save_name = None):
        nb_words = len(X)
        informations = np.zeros(nb_words).astype('float')
        for idx, guess in enumerate(X):
            informations[idx] = self.get_information(X_reduced,guess)
            if True:
                print(f"Processed {self._vec2word(guess)} ({idx+1:>5} of {nb_words}): Expected {informations[idx]:.3f} bits of information")
        
        df = pd.DataFrame()
        df['word'] = self._mat2words(X)
        df['expected_information'] = informations
        df['expected_nb_words'] = [nb_words*(1/2)**info for info in informations]
        
        df = df.sort_values(by = 'expected_information', ascending = False).reset_index()
        
        if save_name:
            timestamp = str(datetime.datetime.today()).split('.')[0].replace('-','_').replace(':','_').replace(' ','_')
            df.to_csv(f"{save_name}_{timestamp}.csv")
        else:
            return df
        
    def register_guess(self, guess, response, verbose = False):
        self.nb_guesses += 1
        guess_word = guess
        guess = list(self._word2vec(guess))
        response = list(response)
        expected_information = self.get_information(self.X, guess)
        old_nb_words = len(self.X_reduced)
        self.X_reduced = self.reduce_vecs(self.X_reduced, guess, response)
        new_nb_words = len(self.X_reduced)
        prob = new_nb_words / old_nb_words
        realized_information = -np.log2(prob)
        expected_nb_words = old_nb_words*(1/2)**expected_information
    
        if verbose:
            print(f"You guessed '{guess_word}'.")
            print(f"Expected information: {expected_information:.3f} bits (leading to {expected_nb_words:.1f} words)")
            print(f"Realized information: {realized_information:.3f} bits (leading to {new_nb_words} words)")
            print('\n')
    
    def test_guess(self, guess):
        expected_information = self.get_information(self.X_reduced, self._word2vec(guess))
        expected_nb_words = len(self.X_reduced)*(1/2)**expected_information
        print(f"Guessing on {guess} is expected to give {expected_information:.3f} bits of information (leading to {expected_nb_words:.1f} words)")
        
    def status(self, show_best_nb = 0):
        wait_message = f"Waiting for guess {self.nb_guesses + 1}..."
        nb_words_message = f"There are {len(self.X_reduced)} words to choose from."
        best_words_message = f"Here are the top {show_best_nb} best words to choose next:"
        
        if self.nb_guesses == 0:
            print(wait_message)
            print(nb_words_message)
            if show_best_nb > 0:
                print("The best words among all words takes long to calculate. Please see stored file.")
        else:
            if self.nb_guesses < 6:
                print(wait_message)
            else:
                print("No more guesses left")
            print(nb_words_message)
            if show_best_nb > 0:
                df = self.get_all_informations(self.X, self.X_reduced)
                print(best_words_message)
                print(df.head(show_best_nb))
                
        print('\n')
        
    # TODO: 
    # def get_response(guess, correct):
    #    ...
    #    return response
    

In [405]:
wb = WordleBrain(all_words)
#wb.get_all_informations(wb.X, save_name = 'word_informations')

In [406]:
wb.register_guess('raset', [2,0,0,1,1], verbose = True)
wb.status(10)

You guessed 'raset'.
Expected information: 7.398 bits (leading to 48.6 words)
Realized information: 8.753 bits (leading to 19 words)


Waiting for guess 2...
There are 19 words to choose from.
Processed abbed (    1 of 8195): Expected 2.236 bits of information
Processed abede (    2 of 8195): Expected 2.184 bits of information
Processed abens (    3 of 8195): Expected 4.613 bits of information
Processed abers (    4 of 8195): Expected 2.805 bits of information
Processed abort (    5 of 8195): Expected 2.807 bits of information
Processed acces (    6 of 8195): Expected 1.485 bits of information
Processed acids (    7 of 8195): Expected 1.466 bits of information
Processed adder (    8 of 8195): Expected 2.036 bits of information
Processed adels (    9 of 8195): Expected 5.125 bits of information
Processed adept (   10 of 8195): Expected 4.056 bits of information
Processed adhds (   11 of 8195): Expected 1.021 bits of information
Processed adled (   12 of 8195): Expected 2.378 bits of inf

KeyboardInterrupt: 

In [393]:
wb.register_guess('retro', [2,1,2,0,0], verbose = True)
wb.status(10)

You guessed 'retro'.
Expected information: 2.603 bits (leading to 3.1 words)
Realized information: 4.248 bits (leading to 1 words)


Waiting for guess 3...
There are 1 words to choose from.
Here are the top 10 best words to choose next:
   index   word  expected_information  expected_nb_words
0      0  rytme                   0.0                1.0




In [385]:
wb.register_guess('rumle', [2,0,1,0,2], verbose = True)
wb.status(10)

You guessed 'rumle'.
Expected information: 2.774 bits (leading to 2.3 words)
Realized information: inf bits (leading to 0 words)


Waiting for guess 4...
There are 0 words to choose from.
Here are the top 10 best words to choose next:
Empty DataFrame
Columns: [index, word, expected_information, expected_nb_words]
Index: []




/Users/jasoba/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:144: RuntimeWarning: divide by zero encountered in log2


In [ ]:
wb.register_guess('bores', [1,0,0,0,0], verbose = True)
wb.status(5)

['ammen',
 'ammer',
 'ammes',
 'ammet',
 'emmen',
 'emmer',
 'emmes',
 'emmet',
 'mamas',
 'mamen',
 'mamer',
 'ommen',
 'ommer',
 'ommes',
 'ømmer',
 'ømmes']

False

In [254]:
all([])

True

In [238]:
wb._mat2words(X[np.all(X[:,[2,3]] == np.array(wb._word2vec('sagde'))[[2,3]], axis = 1)])

['bygde',
 'bygds',
 'hugde',
 'kogde',
 'lagde',
 'lægde',
 'lægds',
 'sagde',
 'sigde',
 'sigds']

In [228]:
[x for x in wb._mat2words(X) if 'sagde' in x]

['sagde']

In [204]:
data = pd.read_csv("ods_fullforms_2020-08-26.csv",sep='\t',header = None)

In [206]:
words = list(data[0])
map_letters = {
        'aa':'å',
        'ã':'a',
        'ê':'e',
        'é':'e'
        }
for old, new in map_letters.items():
    words = [w.replace(old, new) for w in words]
    
    
# 5 letters
words = [w for w in words if len(w) == 5]

# Remove words with strange symbols
only_allowed = "abcdefghijklmnopqrstuvxyzæøå"
word_allowed = lambda word: all([letter in only_allowed for letter in word])
words = [w for w in words if word_allowed(w)]

# Drop duplicates
words = sorted(list(set(words)))

[w for w in words if 'agde' in w]

['lagde', 'sagde']

In [180]:
~(X[:,3] == 4)

array([False, False,  True, ...,  True,  True,  True])

In [198]:
def get_information(all_words, guess):
    nb_all_words = len(all_words)
    probs = []
    information = 0
    # For all patterns
    for idx, response in enumerate(list(itertools.product(*[[0,1,2] for _ in range(5)]))):
        nb_reduced = len(update_words(all_words, guess, response))
        prob = nb_reduced / nb_all_words
        information += - prob * np.log2(prob) if prob != 0 else 0

    return information

In [201]:
%prun get_information(all_words,'hbupæ')

         2015 function calls in 0.492 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      314    0.276    0.001    0.276    0.001 <ipython-input-197-2719664d8c6d>:26(<listcomp>)
      314    0.115    0.000    0.115    0.000 <ipython-input-197-2719664d8c6d>:6(<listcomp>)
      314    0.089    0.000    0.089    0.000 <ipython-input-197-2719664d8c6d>:15(<listcomp>)
      243    0.010    0.000    0.491    0.002 <ipython-input-197-2719664d8c6d>:1(update_words)
        1    0.002    0.002    0.492    0.492 <ipython-input-198-b3171ce83484>:1(get_information)
      581    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
      244    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        1    0.000    0.000    0.492    0.492 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 <ipython-input-198-b3171ce83484>:6(<listcomp>)
        1    0.000    0.000    0.492    0.492 <

In [195]:
informations = [get_information(all_words,guess) for guess in list(all_words)[:10]]


KeyboardInterrupt: 

In [188]:
df = pd.DataFrame()
df['ord'] = list(all_words)[:100]
df['information'] = informations
df.sort_values(by = 'information', ascending = False)

,ord,information
96,seret,7.011466
56,skeet,6.113415
50,beter,6.021064
92,tilse,5.696584
24,samer,5.684311
...,...,...
14,opgiv,3.577537
75,opråb,3.529832
0,påbid,3.350558
79,omhug,3.269102


In [182]:
get_information(all_words,'skeet')

6.113414650810543

In [137]:
print(len(all_words))
words = update_words(all_words, 'bores',[0,0,0,1,1])
print(len(words))
words = update_words(words, 'liste',[0,0,1,0,2])
print(len(words))
words = update_words(words, 'snude',[2,0,0,0,2])
print(len(words))
words = update_words(words, 'smage',[2,0,0,0,2])
print(len(words))
words = update_words(words, 'svæve',[2,0,0,0,2])
print(len(words))
words

11729
645
173
34
14
6


['sykke', 'skøje', 'sekse', 'skeje', 'søkke', 'sejse']

In [132]:
[w for w in all_words if 'skeje' in w]

['skeje']